In [4]:
import pandas as pd 
df = pd.read_csv(r"C:\Users\Rasulbek907\Desktop\Project_One\Data\Raw_Data\titanic.csv")

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


# Missing Values 

In [6]:
def missing_val(df):
    for col in df.columns:
        if df[col].isnull().sum():
            if df[col].dtype == "object":
                df[col] = df[col].fillna(df[col].mode()[0])
            else:
                df[col] = df[col].fillna(df[col].mean())
    return df 

In [7]:
df = missing_val(df)

# Feature Enginering

In [8]:
df["FamilySize"] = df["SibSp"] + df["Parch"] + 1 

In [9]:
df["IsAlone"] = (df["FamilySize"] == 1).astype(int)

In [10]:
df["Title"] = df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)

<>:1: SyntaxWarning: invalid escape sequence '\.'
<>:1: SyntaxWarning: invalid escape sequence '\.'
C:\maya_temp\ipykernel_13848\3631293841.py:1: SyntaxWarning: invalid escape sequence '\.'
  df["Title"] = df["Name"].str.extract(' ([A-Za-z]+)\.', expand=False)


In [11]:
df["Title"] = df["Title"].replace(['Lady', "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Rare", "Dona"])
df['Title'] = df["Title"].replace({"Mlle": "Miss", "Ms":"Miss", "Mme":"Mrs"})

df.drop(columns=["Name"], inplace=True)

C:\maya_temp\ipykernel_13848\2873047962.py:1: FutureWarning: Series.replace without 'value' and with non-dict-like 'to_replace' is deprecated and will raise in a future version. Explicitly specify the new values instead.
  df["Title"] = df["Title"].replace(['Lady', "Countess", "Capt", "Col", "Don", "Dr", "Major", "Rev", "Sir", "Jonkheer", "Rare", "Dona"])


In [12]:
import numpy as np
df["Fare_Log"] = np.log1p(df["Fare"])
df.drop(columns=["Fare"], inplace=True)

# Feature Selection

# Filtering Usuli bo`yicha

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    object 
 8   Cabin        891 non-null    object 
 9   Embarked     891 non-null    object 
 10  FamilySize   891 non-null    int64  
 11  IsAlone      891 non-null    int64  
 12  Title        891 non-null    object 
 13  Fare_Log     891 non-null    float64
dtypes: float64(2), int64(7), object(5)
memory usage: 97.6+ KB


In [14]:
for col in df.select_dtypes(include=["object", "category"]):
    df[col] = df[col].astype("category").cat.codes

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int8   
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    int16  
 8   Cabin        891 non-null    int16  
 9   Embarked     891 non-null    int8   
 10  FamilySize   891 non-null    int64  
 11  IsAlone      891 non-null    int64  
 12  Title        891 non-null    int8   
 13  Fare_Log     891 non-null    float64
dtypes: float64(2), int16(2), int64(7), int8(3)
memory usage: 68.9 KB


In [16]:
import pandas as pd 
import plotly.express as px 

numeric_cols = df.select_dtypes(
    include=["float64", "int64", "int16", "int8"]
).columns

corr_matrix = df[numeric_cols].corr()

survived_corr = (
    corr_matrix[["Survived"]]
    .sort_values(by="Survived", ascending=False)
)

heatmap_data = survived_corr.reset_index()
heatmap_data.columns = ["Feature", "Correlation"]

fig = px.bar(
    heatmap_data,
    x="Feature",
    y="Correlation",
    title="Survived bilan feature'lar korrelyatsiyasi",
    labels={
        "Correlation": "Korrelyatsiya qiymati",
        "Feature": "Feature nomi"
    }
)

fig.show()

In [17]:
import plotly.express as px

numeric_cols = df.select_dtypes(
    include=["float64", "int64", "int16", "int8"]
).columns

corr_matrix = df[numeric_cols].corr()

fig = px.imshow(
    corr_matrix,
    text_auto=".2f",
    aspect="equal",               
    color_continuous_scale="RdBu",
    zmin=-1,
    zmax=1,
    title="Feature'lar o‘rtasidagi korrelyatsiya (Correlation Matrix)"
)

fig.update_layout(
    xaxis_title="Feature'lar",
    yaxis_title="Feature'lar"
)

fig.show()


# Higly Correlated 

In [34]:
import numpy as np 

corr_matrix = df.corr().abs()
upper = corr_matrix.where(
    np.triu(np.ones(corr_matrix.shape), k=1).astype(bool)

)

to_drop = [col for col in upper.columns if any(upper[col] > 0.6)]
print("Olib Tashlangan Featurelar: ", to_drop)
df_reduced = df.drop(columns=to_drop)

Olib Tashlangan Featurelar:  ['FamilySize', 'IsAlone', 'Fare_Log']


In [24]:
df_reduced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Sex          891 non-null    int8   
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Ticket       891 non-null    int16  
 8   Cabin        891 non-null    int16  
 9   Embarked     891 non-null    int8   
 10  FamilySize   891 non-null    int64  
 11  IsAlone      891 non-null    int64  
 12  Title        891 non-null    int8   
 13  Fare_Log     891 non-null    float64
dtypes: float64(2), int16(2), int64(7), int8(3)
memory usage: 68.9 KB


# Low Varience

In [35]:
import numpy as np
import pandas as pd 
from sklearn.feature_selection import VarianceThreshold

x = df.drop(columns=["Survived"])
y = df["Survived"]

x = x.select_dtypes(include=["int64", "int16", "int8", "float64"])

print("Boshlang`ich shape: ", x.shape)


threshold = 0.30
var_selector = VarianceThreshold(threshold=threshold)
var_selector.fit(x)

low_varience_features = x.columns[~var_selector.get_support()]

print("Kam Tasirli featurlear")
print(list(low_varience_features))

x_var_filtered = x.drop(columns=low_varience_features)

print("Tashlab yuborilgandan keyingi featurlar", x_var_filtered.shape)

Boshlang`ich shape:  (891, 13)
Kam Tasirli featurlear
['Sex', 'IsAlone']
Tashlab yuborilgandan keyingi featurlar (891, 11)


In [38]:
x_var_filtered.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_One\Data\Feature_Selection_Data\Feature_Filtering_Selection.csv")

# Feature Selection Wrapper

# RFE + LogisticRegression

# RFE - Recursive Feature Elimination - Featurelarni bosqichma bosqich yo`q qilib borish 

In [39]:
from sklearn.feature_selection import RFE 
from sklearn.linear_model import LogisticRegression

x = df.drop(columns=["Survived"])
y = df["Survived"]

x = x.select_dtypes(include=["int64", "int16", "int8", "float64"])

print("Boshlang`ich shape: ", x.shape)

model = LogisticRegression(
    max_iter=1000,
    solver="liblinear"
)

rfe = RFE(
    estimator=model,
    n_features_to_select=10
)

rfe.fit(x, y)

selected_features = x.columns[rfe.support_]
print("Tanlangan Featurelar: " )
print(list(selected_features))

x_rfe = x[selected_features]

print("Final shape after rfe selection: ", x_rfe.shape)

Boshlang`ich shape:  (891, 13)
Tanlangan Featurelar: 
['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Embarked', 'FamilySize', 'IsAlone', 'Title', 'Fare_Log']
Final shape after rfe selection:  (891, 10)


# RFE + Decision Tree

In [40]:
from sklearn.feature_selection import RFE 
from sklearn.tree import DecisionTreeRegressor

x = df.drop(columns=["Survived"])
y = df["Survived"]

x = x.select_dtypes(include=["int64", "int16", "int8", "float64"])

print("Boshlang`ich shape: ", x.shape)

model = DecisionTreeRegressor(
    random_state=42,
    max_depth=5
)

rfe_dt = RFE(
    estimator=model,
    n_features_to_select=10
)

rfe_dt.fit(x, y)

selected_features = x.columns[rfe_dt.support_]
print("Tanlangan Featurelar: " )
print(list(selected_features))

x_rfe = x[selected_features]

print("Final shape after rfe selection: ", x_rfe.shape)

Boshlang`ich shape:  (891, 13)
Tanlangan Featurelar: 
['PassengerId', 'Pclass', 'Sex', 'Age', 'Ticket', 'Cabin', 'FamilySize', 'IsAlone', 'Title', 'Fare_Log']
Final shape after rfe selection:  (891, 10)


# Feature Selectin Embedded 

# Embedded + Lasso Regression

In [41]:
import pandas as pd 
import numpy as np 

from  sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

x = df.drop(columns=["Survived"])
y = df["Survived"]

x = x.select_dtypes(include=["int64", "int16", "int8", "float64"])

print("Boshlang`ich shape: ", x.shape)

lasso_pipeline = Pipeline(steps=[
    ("scaler", StandardScaler()),
    ("lasso", Lasso(alpha=0.01, random_state=42))
])

lasso_pipeline.fit(x, y)

lasso_model = lasso_pipeline.named_steps["lasso"]

lasso_coef = pd.Series(
    lasso_model.coef_,
    index=x.columns
)

selected_features = lasso_coef[lasso_coef != 0].index

print("Lasso bilan tanlangan featurelar: ")
print(list(selected_features))


x_lasso = x[selected_features]

print("Selected features after Lasso ", x_lasso.shape)

Boshlang`ich shape:  (891, 13)
Lasso bilan tanlangan featurelar: 
['Pclass', 'Sex', 'Age', 'SibSp', 'Ticket', 'Cabin', 'Embarked', 'FamilySize', 'IsAlone', 'Title', 'Fare_Log']
Selected features after Lasso  (891, 11)


# Embedded + Random Forest

In [42]:
import pandas as pd 
import numpy as np 
from sklearn.ensemble import RandomForestRegressor

x = df.drop(columns=["Survived"])
y = df["Survived"]

x = x.select_dtypes(include=["int64", "int16", "int8", "float64"])

print("Boshlang`ich shape: ", x.shape)

rf_model = RandomForestRegressor(
    n_estimators=300,
    random_state=42,
    n_jobs=1
)

rf_model.fit(x, y)

rf_importance = pd.Series(
    rf_model.feature_importances_,
    index = x.columns
).sort_values(ascending=False)


print("RandomForestRegressor feature importance ")
print(rf_importance)

threshold = 0.01 

selected_features = rf_importance[rf_importance > threshold].index


print("RandomForestRegressor feature seelction")
print(list(selected_features))


x_rf = x[selected_features]

print("Selected features after Lasso ", x_rf.shape)

Boshlang`ich shape:  (891, 13)
RandomForestRegressor feature importance 
Sex            0.296739
Ticket         0.146301
PassengerId    0.117420
Age            0.105575
Fare_Log       0.105472
Pclass         0.083801
Cabin          0.041079
FamilySize     0.033518
Title          0.024827
SibSp          0.022757
Embarked       0.014348
Parch          0.005730
IsAlone        0.002433
dtype: float64
RandomForestRegressor feature seelction
['Sex', 'Ticket', 'PassengerId', 'Age', 'Fare_Log', 'Pclass', 'Cabin', 'FamilySize', 'Title', 'SibSp', 'Embarked']
Selected features after Lasso  (891, 11)


In [45]:
x_rf.to_csv(r"C:\Users\Rasulbek907\Desktop\Project_One\Data\Feature_Selection_Data\Feature_Selection_Final.csv")

# Feature Selection  -- featurelearni Tanlab olish bosqichi

# Feature Selection 3 ta qadamga bo`linadi 
- 1-tur = Feature Filtering
- 2-tur = Feature Wrapper
- 3-tur = Feature Embedded

# Feature Filtering



-  statistical qiymatlarga ko`ra featurelarni tanlanadi
- featureni qanchalik target qiymatga bog`liqligiga ko`ra tanlanadi // agarda 1 bo`lsa = Strong Relationship // agarda 0  bo`lsa = independet relationship - bog`liqlik yo`q


# Feature Filtering Metodlari : Correlation, Variance threshold, Anova

# Feature Filteringda 2 xil natija ko`p kuzatilishi mumkin

- 1- Multicollinearity -- juda yuqori bo`gliq bo`lgan feature
- 2- LoW Varience -- juda kam tasir qilayotgan feature 

# Feature Filtering ni + va  - tamonlari

- "+" lari 
- tez ishlaydi  (algaritm va  training qilish shart emas )
- high -dimensional datasetlar bilan yaxshi ishlaydi 
- overfittingni kamaytiradi 


- "-" lari 
- barcha featurelar alohida alohida baholanadi
- kafolatli emas  

# Feature Wrapper 

# Bu usulda - ML model training qilgandan so`ng featurelarni baxolaydi 

# Baxolash mezoni - qanchalik model accuracyga qo`shgan xissasiga ko`ra tanlanadi - relative contributions

# Feature Wrapperning bir nechta metodlari bor -- RFE, Forward Selection, Backward Elimination

# Feature Wrapperning ishlashi-- Eng avval : turli xil datasetlar bilan ko`p marotaba tarining qiladi  , Keyin esa: eng yuqori natija bergan subsetlar eng yuqori featurelar sifatida qabul qilinadi

# Feature  Wrapperning + va - tamonlari
- "+" lari 
- featurelarni alohidadan ko`ra guruh bilan yaxshi natija berishini xisobga oladi
- odatda fiteringdan  ko`ra yaxshiroq natija beradi 

- "- " lari 
- ko`p vaqt oladi 
- overfittingga moyil 

# Feature Embedded usuli

# bu usulda model training davomida qo`shgan hissasiga qarab  featurelarni tanlab oladi 

# ishlash tartibi - feature  qanchalik  impurity ni kamaytirganiga qarab aniqlanadi (% larda o`lchanadi )
- impurity bu modeldagi shovqin 

# Feature Embedded Metodlari: Lasso Regression, gradient Boosting, Random Forest 

# Feature Embedded ning + va  -  tamonlari 
- "+" lari 
- featurelar training davomida eng yaxshilari tanlanadi
- featurearni birgalikda muhimligio inobatga olinadi 
- katta datasetlar bilan yaxshi ishlaydi 

- "-" lari 
- interpretability qiyin
- scaling uchun bazi metodlar sensitive  

# XULOSA

# Qaysi xolatda qaysi usul bilan  ishlanadi

- FILTERING     

- high dimensional datasets
- tezkor feature selection uchun 
- model trainingsiz va oshiqcha vaqtlarsiz 


- WRAPPER 

- kichik yoki o`rta hajmli datasetlar
- eng yuqori natija kerak bo`lsa 
- ko`p vaqt bor bo`lsa 


- EMBEDDED 

-  avtomatik feature selection 
- o`rtacha dataset 
- xar bir algoritm uchun  maxsus featurelar kerak bo`lsa 

# Korrelyatsiya - bu ikki o`zgaruvchining biur biriga qanchalik bo`gliqligini ko`rsatuvchi o`lchov 

# Range -1 dan +1 gacha 

agarda +1 bo`glanish bo`lsa bu ijobiy bog`lanish deyiladi 
agarda -1 bo`glanish bo`lsa bu salbiy bog`lanish deyiladi 
agarda 0 bo`lsa bog`lanish yo`q deyiladi

# Highlty correlated features - bir biriga juda yaqin bo`glangan featurelardir (r >= 0.8)

# Higly correlated bo`lsa nima muammolar kelib chiqadi
- 1-Multicollinearity hosil bo`ladi 
- 2- modelning interpretatsiyasi qiyinlashadi
- 3- Bazi ML algoritmlari (Linear Regression, Logistic regression, uchun xatolikni oshiradi)

# Low Correlated - bir biriga juda kam tasir qilayotgan featurelar

# Low Correlated da Multicollinearity muammosi yo`q 

# Yuqoridagilardan kleib chiqgan holda ML algoritmga qarab qaror qabul qiling 
- Tree based (Random Forest, XGBoost ) -->    multicollinearity muammo emas 
- Linear Algoritmlar (Logistic Regression Linear regression) --->  multicollinearity muammo qiladi 